# Visualizing RMS Grid datasets in the sandbox

First we import the necessary modules:

In [ ]:
import os,sys
#set the path to the Sandbox repository
sys.path.append('../../sandbox')
#import the sandbox
import sandbox as sb

### set up the sandbox
Next we initialize the sandbox components for the projector and the sensor. To do so we need to specify a calibration data json file. If you do not have a calibration yet (or if the projected image of the sandbox does not align with your sirface anymore), create a new calibration first by following the instructions in the "calibration.ipynb" notebook.

In [ ]:
#load the calibration data from the json file
calib = sb.CalibrationData(file='my_calibration.json')

In [ ]:
#instatiate a projector instance:
# a new window will open. drag it to the projector screen and maximize by pressing F11

projector = sb.Projector(calib)

In [ ]:
#initiate the kinect sensor:

sensor = sb.KinectV2(calib)

#if you do not have a kinect sensor connected:
#comment out the line above and uncomment the line below to use a dummy sensor.

#sensor = sb.DummySensor(calib, depth_limits=(500,2000))

## Block Module
The Sandbox.BlockModule class contains all necessary methods to visualize RMS grid data. In the following steps we will create an instance of the module and load one or more RMS grids from VIP Datasets. Furthermore we will call methods to specify a custom colorbar and range for some datasets. 
For now the dimensions of the RMS grid are scaled to the dimensions of the sandbox, so if you need for example a larger vertical block size, adjust the vertical range accordingly. The horizontal aspect ratio of the sandbox is 1.2:1. Any model with a similar aspect ratio should look just fine. 

In [ ]:
#instantiate the block module module to display RMS Grids:
module = sb.BlockModule(calib, sensor, projector)

In [ ]:
# specify path to the folder where the grids are stored: 
grid_folder='/Users/Admin/PycharmProjects/open_AR_Sandbox/grids_do_not_push/'
module.load_model(grid_folder+"VIP_for_UKA_linear.pickle")

In [ ]:
# specify custom colormaps:
#norm specifies the minimum and maximum value of the cells
module.set_colormap('fluid',cmap=module.create_cmap(['blue','green','red']),norm=module.create_norm(0,2))
module.set_colormap('ntg',cmap=module.create_cmap(['green','yellow']),norm=module.create_norm(0.0,1.0))

# for all other datasets, the 'jet' colormap is set as default:  
module.set_colormaps()

## start the sandbox
now your model is set and ready to go! There are only two things to do: 
1. call the setup method of the model that prepares the sandbox 
2. call module.run() to start the sandbox thread. 

In [ ]:
#setup the module to prepare the "run" coroutine :
module.setup() 

In [ ]:
# start the sandbox thread
module.run()

#Done! The Sandbox should now run

## manipulation in runtime
the sandbox is running in a separate thread. that means that you can make (some) changes while the sandbox is running. One build in example is the dataset selector widget that you can use to switch between the visualized datasets in runtime.

For all other changes you make to any parameters it is important that you acquire and release the module's thread lock object (module.lock) each time that you want to make any changes. This way you make sure that you do not write any data that the sandbox is currently trying to access (which might lead to some unexpected behaviour).

Here are some examples of changes you can make: 

In [ ]:
# show a selector widget to switch between the currently shown datasets:

module.show_widgets()

In [ ]:
# change the gaussian filter parameter of the sensor

with module.lock:
    print("sigma gauss was: "+str(sensor.sigma_gauss) )
    sensor.sigma_gauss=5
    

In [ ]:
# change the colormap of a dataset

with module.lock:
    module.set_colormap('fluid',
                        cmap=module.create_cmap(['red','white','green']),
                        norm=module.create_norm(0,2)
                       )

## stop the sandbox
to stop the sandbox from running you have two options: Either restart the kernel or manually terminate the sandbox thread with module.stop()

Using the stop() method has two advantages: all your variables  are still there and you can restart your sandbox any time you like by calling module.run() again.

In [ ]:
# execute this cell to stop the sandbox thread:
module.stop()